In [153]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

In [154]:

# Загружаем данные
df = pd.read_csv('data/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков шесть — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

- Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой.
- Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра.

Целевой переменной является столбец RainTomorrow. Значение этой переменной мы и будем пытаться предсказать.

- Date — дата, в которую зафиксировано наблюдение;
- Location — местонахождение метеорологической станции;
- MinTemp — минимальная температура (℃);
- MaxTemp — максимальная температура (℃);
- Rainfall — количество осадков (дождь) за сутки (мм);
- Evaporation — количество испарений до 9 утра (мм);
- Sunshine — количество часов в сутках, когда светило солнце;
- WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
- WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
- WindDir9am — направление ветра в 9 утра;
- WindDir3pm — направление ветра в 3 часа дня;
- WindSpeed9am — скорость ветра в 9 часов утра;
- WindSpeed3pm — скорость ветра в 3 часа дня;
- Humidity9am — влажность в 9 утра;
- Humidity3pm — влажность в 3 часа дня;
- Pressure9am — атмосферное давление в 9 утра;
- Pressure3pm — атмосферное давление в 3 часа дня;
- Cloud9am — часть неба, закрытая облаками, в 9 утра;
- Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
- Temp9am — температура в 9 утра;
- Temp3pm — температура в 3 часа дня;
- RainToday — наличие дождя в этот день;
- RainTomorrow — наличие дождя на следующий день.

In [156]:
# Количество пропусков в каждом столбце
missing_values_per_column = df.isnull().sum()
print("Пропуски по столбцам:\n", missing_values_per_column)

# Общее количество пропусков в датафрейме
total_missing_values = df.isnull().sum().sum()
print(f"Всего пропущенных значений в датафрейме: {total_missing_values}")


Пропуски по столбцам:
 Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64
Всего пропущенных значений в датафрейме: 343248


В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?

In [157]:
# Определяем порог 40% пропущенных значений
threshold = 0.4 * len(df)

# Находим признаки, в которых количество пропусков превышает порог
columns_to_drop = df.columns[df.isnull().sum() > threshold]

# Количество удалённых признаков
num_dropped_columns = len(columns_to_drop)

# Удаляем признаки
df_cleaned = df.drop(columns=columns_to_drop)

print(f"Удалено {num_dropped_columns} признаков: {list(columns_to_drop)}")


Удалено 3 признаков: ['Evaporation', 'Sunshine', 'Cloud3pm']


Теперь обработаем признаки RainToday и RainTomorrow таким образом, чтобы вместо yes было значение 1, а вместо no — значение 0. Обратите внимание на то, что в признаках RainToday и RainTomorrow присутствуют пропуски, и их трогать не нужно, они должны остаться пропусками. Поэтому обрабатывайте столбцы таким образом, чтобы не видоизменить пропущенные значения.

Вычислите среднее арифметическое для преобразованного признака RainToday и запишите его в ответ, предварительно округлив до двух знаков после точки-разделителя.

In [158]:
# Меняем 'Yes' на 1, 'No' на 0, оставляя NaN неизменными
df.RainToday = df.RainToday.map({'No': 0, 'Yes': 1})
df.RainTomorrow = df.RainTomorrow.map({'No': 0, 'Yes': 1})

In [159]:
mean_rain_today = df['RainToday'].mean()
print(f"Среднее значение RainToday: {mean_rain_today:.2f}")


Среднее значение RainToday: 0.22


Обработайте признак Date таким образом, чтобы выделить в отдельный признак Month (номер месяца). Изначальный признак Date удалите. Определите, какой месяц имеет самую большую часть дождливых дней относительно всех дней месяца. В качестве ответа введите порядковый номер месяца.

In [160]:
df.Date = pd.to_datetime(df.Date)
df['Month'] = df.Date.dt.month
df.drop('Date', axis = 1, inplace = True)
df_season = df.groupby('Month').mean(numeric_only=True)
df_season[['RainToday']]

,RainToday
Month,
1,0.189484
2,0.206746
3,0.217135
4,0.216845
5,0.222163
6,0.263638
7,0.270736
8,0.253167
9,0.229135


2. Определяем месяц с наибольшей долей дождливых дней

In [161]:
# Считаем количество дождливых дней (`RainToday == 1`) по месяцам
rainy_days_per_month = df.groupby('Month')['RainToday'].sum()

# Считаем общее количество дней по месяцам
total_days_per_month = df['Month'].value_counts().sort_index()

# Рассчитываем долю дождливых дней
rain_ratio = rainy_days_per_month / total_days_per_month

# Определяем месяц с максимальной долей
rainiest_month = rain_ratio.idxmax()

print(f"Месяц с самой большой долей дождливых дней: {rainiest_month}")


Месяц с самой большой долей дождливых дней: 7


Обработайте оставшиеся категориальные признаки. С помощью метода get_dummies с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

Кодировку признаков важно выполнить именно в следующем порядке: categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']. Это необходимо для того, чтобы ваши дальнейшие ответы сходились с нашим решением, так как алгоритм случайного леса, который мы будем использовать в дальнейшем, чувствителен к порядку столбцов. Аргумент categoricals передаётся в функцию pd.get_dummies() с использованием ключевого слова columns, а именно columns=categoricals.

Сколько теперь признаков в данных, если считать целевую переменную?

In [162]:
categoricals = ['Month', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']
df_dummies = pd.get_dummies(df, columns=categoricals)
# Определяем количество признаков (с целевой переменной)
df_dummies.shape


(145460, 127)

Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмите число 31.

Каково среднее значение целевой переменной на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [163]:
df_cleaned = df_dummies.dropna()

from sklearn.model_selection import train_test_split

# Определяем признаки (X) и целевую переменную (y)
X = df_cleaned.drop(columns=['RainTomorrow'])  # Удаляем целевой признак из данных
y = df_cleaned['RainTomorrow']  # Целевая переменная

# Разбиваем данные на 70% обучающих и 30% тестовых
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)


In [164]:
mean_target_test = y_test.mean()
print(f"Среднее значение целевой переменной на тестовой выборке: {mean_target_test:.2f}")


Среднее значение целевой переменной на тестовой выборке: 0.22


Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp). Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используйте np.random.randint(): сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. Случайность фиксируйте с помощью np.random.seed(31).

Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений. Ответ округлите до двух знаков после точки-разделителя.

In [165]:
import numpy as np

# Фиксируем случайность
np.random.seed(31)

# Размер обучающей выборки
train_size = len(X_train)

# Список для хранения средних значений
bootstrap_means = []

# Генерация 1000 бутстреп-выборок и расчёт средних значений
for _ in range(1000):
    sample_indices = np.random.randint(0, train_size, train_size)  # Генерация случайных индексов
    sample_min_temp = X_train.iloc[sample_indices]['MinTemp']  # Извлекаем данные по индексу
    bootstrap_means.append(sample_min_temp.mean())  # Вычисляем среднее

# Оценка стандартного отклонения
std_dev = np.std(bootstrap_means)
print(f"Стандартное отклонение бутстреп-средних: {std_dev:.2f}")


Стандартное отклонение бутстреп-средних: 0.03


Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее.

В качестве ответа введите значение метрики roc_auc на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [166]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [167]:
# Создаём модель логистической регрессии с параметрами по умолчанию
log_reg = LogisticRegression()

# Обучаем модель
log_reg.fit(X_train, y_train)


C:\Users\fkara\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [168]:
# Получаем вероятности предсказаний
y_proba = log_reg.predict_proba(X_test)[:, 1]  # Берём вероятности положительного класса (RainTomorrow = 1)

# Вычисляем ROC AUC
roc_auc = roc_auc_score(y_test, y_proba)

print(f"ROC AUC на тестовой выборке: {roc_auc:.2f}")


ROC AUC на тестовой выборке: 0.88


Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. С помощью GridSearchCV сделайте перебор гиперпараметров по следующей сетке: params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]} Для параметра кросс-валидации cv задайте значение 3. Для решающего дерева определите параметр random_state=42. Остальные параметры оставьте по умолчанию. 1. Вычислите значение roc_auc для решающего дерева с гиперпараметрами, определёнными в качестве оптимальных. Ответ округлите до двух знаков после точки-разделителя.

In [169]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score


In [170]:
# Определяем сетку гиперпараметров
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5, 7, 9, 11]}

# Инициализируем модель решающего дерева
tree_clf = DecisionTreeClassifier(random_state=42)

# Запускаем GridSearchCV с кросс-валидацией (cv=3)
grid_search = GridSearchCV(tree_clf, params, cv=3, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 7, 9, 11],
                         'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4]},
             scoring='roc_auc')

In [171]:
best_params = grid_search.best_params_
print(f"Оптимальные параметры: {best_params}")


Оптимальные параметры: {'max_depth': 5, 'max_leaf_nodes': 9, 'min_samples_split': 2}


In [172]:
# Получаем вероятности предсказаний на тестовой выборке
y_proba = grid_search.best_estimator_.predict_proba(X_test)[:, 1]

# Вычисляем roc_auc
roc_auc = roc_auc_score(y_test, y_proba)

print(f"ROC AUC для оптимальной модели: {roc_auc:.2f}")


ROC AUC для оптимальной модели: 0.83


К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей.

Теперь постройте случайный лес, включающий 100 деревьев. Задайте параметр random_state=31. Остальные параметры оставьте по умолчанию.

Какой теперь будет метрика roc_auc на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [173]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


In [174]:
# Создаём модель случайного леса с 100 деревьями и random_state=31
rf_clf = RandomForestClassifier(n_estimators=100, random_state=31)

# Обучаем модель
rf_clf.fit(X_train, y_train)


RandomForestClassifier(random_state=31)

In [175]:
# Получаем вероятности предсказаний на тестовой выборке
y_proba = rf_clf.predict_proba(X_test)[:, 1]  # Вероятности класса `RainTomorrow = 1`

# Вычисляем ROC AUC
roc_auc = roc_auc_score(y_test, y_proba)

print(f"ROC AUC для случайного леса: {roc_auc:.2f}")


ROC AUC для случайного леса: 0.90


Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие:'max_features', 'min_samples_leaf', 'max_depth'.

Возьмите случайный лес из 100 деревьев и найдите оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая:

{'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}


Перебор осуществите с помощью GridSearchCV. Для параметра кросс-валидации cv задайте значение 3. Случайности фиксируйте параметром random_state = 31. Остальные значения оставьте по умолчанию.

Какое значение roc_auc получилось для оптимальных гиперпараметров?

In [176]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


In [177]:
# Создаём модель случайного леса с 100 деревьями и random_state=31
rf_clf = RandomForestClassifier(n_estimators=100, random_state=31)

# Обучаем модель
rf_clf.fit(X_train, y_train)


RandomForestClassifier(random_state=31)

In [178]:
# Получаем вероятности предсказаний на тестовой выборке
y_proba = rf_clf.predict_proba(X_test)[:, 1]  # Вероятности класса `RainTomorrow = 1`

# Вычисляем ROC AUC
roc_auc = roc_auc_score(y_test, y_proba)

print(f"ROC AUC для случайного леса: {roc_auc:.2f}")


ROC AUC для случайного леса: 0.90


Как мы говорили в предыдущем юните, благодаря случайному лесу можно узнать, какие признаки оказывают большее влияние на целевую переменную по сравнению с другими.

Оцените значимость признаков. Отметьте три признака, которые дают наибольший вклад в целевую переменную (в каждой колонке находится как минимум один верный вариант ответа):

In [179]:
import pandas as pd

# Получаем важность признаков
feature_importances = rf_clf.feature_importances_

# Создаём DataFrame для удобства отображения
importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Сортируем по значимости
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Выводим три самых значимых признака
top_features = importance_df.head(3)
print(top_features)


        Feature  Importance
9   Humidity3pm    0.123994
4      Sunshine    0.091093
11  Pressure3pm    0.055495
